In [1]:
import importlib

In [2]:

import gc
import sys
import time
sys.path.append("/home/shaow/jupyter_base/pycharm_001")
base = "/home/shaow/jupyter_base/pycharm_001"
import torch
import experiment.model.dglBertSemModel as models
import graph_process as G
import obj.GraphBase as GB
import utils

In [3]:
importlib.reload(G)
importlib.reload(GB)

appid_edge, appid_ros_edge, appid_app_edge, app_ros_edge, app_kw_edge, \
           id_index_map ,index_id_map, index_keyword_map, index_ros_map, index_app_map , \
           app_dict = G.process_year_graph(2019)



gb_2019 = GB.GraphBase(appid_edge, appid_ros_edge, appid_app_edge, app_ros_edge, app_kw_edge,
           id_index_map, index_id_map, index_ros_map, index_app_map, index_keyword_map ,
           app_dict)

8.0.19
10
socket 10.0.82.237:3306
done init 3564 nodes from apply id
244800
done init 615669 nodes from application


In [4]:
# importlib.reload(G)
gb = gb_2019.get_split_appid_map('F0301')
t_data = G.get_graph_edges(gb)

(2, 17)
<class 'int'>
<class 'numpy.int64'>
(2, 1079)
<class 'int'>
<class 'numpy.int64'>
(2, 1079)
<class 'int'>
<class 'numpy.int64'>
(2, 4968)
<class 'int'>
<class 'numpy.int64'>


In [5]:
train_data = t_data.train
valid_data = t_data.validation
test_data = t_data.test
# num_rels = t_data.num_rels
num_nodes = gb.node_count()
num_rels = gb.rel_count()

In [6]:

test_graph, test_rel, test_norm = utils.build_test_graph(
        num_nodes, num_rels, train_data)
test_deg = test_graph.in_degrees(
            range(test_graph.number_of_nodes())).float().view(-1,1)
test_node_id = torch.arange(0, num_nodes, dtype=torch.long).view(-1, 1)
test_rel = torch.from_numpy(test_rel)
test_norm = models.node_norm_to_edge_norm(test_graph, torch.from_numpy(test_norm).view(-1, 1))

Test graph:


/home/shaow/jupyter_base/pycharm_001/utils.py:130: RuntimeWarning: divide by zero encountered in true_divide
  norm = 1.0 / in_deg


In [7]:
test_data = torch.from_numpy(test_data)
use_cuda = torch.cuda.is_available()

In [8]:
if use_cuda:
    torch.cuda.set_device('cuda:0')

In [9]:
importlib.reload(models)
model = models.LinkPredict(
                        num_nodes,
                        768,
                        num_rels,
                        gb.ikm, gb.irm,
                        gb.iam, gb.application_dict,
                        #="number of weight blocks for each relation"
                        pretrain_list=[True, False, True]
                        ,
                        num_bases=100,
                        # number of propagation rounds
                        # 3 because the trans road
                        num_hidden_layers=3,
                        dropout=0.2,
                        use_cuda=use_cuda,
                        # regularization weight
                        reg_param=0.01)

init pretrain keywords
init pretrain application
True
all word amount is 2411


In [10]:
if use_cuda:
    model.cuda()

# build adj list and calculate degrees for sampling
adj_list, degrees = utils.get_adj_and_degrees(num_nodes, train_data)

# optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model_state_file = 'model_state.pth'
forward_time = []
backward_time = []

# training loop
print("start training...")

epoch = 0
best_mrr = 0

start training...


In [11]:
while True:
    model.train()
    epoch += 1
    # perform edge neighborhood sampling to generate training graph and data
    g, node_id, edge_type, node_norm, data, labels = \
        utils.generate_sampled_graph_and_labels(
            train_data, sample_size=500, split_size=0.5,
            num_rels=num_rels, adj_list=adj_list, degrees=degrees, negative_rate=5,
            sampler='uniform')
    # print("Done edge sampling")
    # set node/edge feature
    node_id = torch.from_numpy(node_id).view(-1, 1).long()
    edge_type = torch.from_numpy(edge_type)
    edge_norm = models.node_norm_to_edge_norm(g, torch.from_numpy(node_norm).view(-1, 1))
    data, labels = torch.from_numpy(data), torch.from_numpy(labels)
    deg = g.in_degrees(range(g.number_of_nodes())).float().view(-1, 1)
    if use_cuda:
        node_id, deg = node_id.cuda(), deg.cuda()
        edge_type, edge_norm = edge_type.cuda(), edge_norm.cuda()
        data, labels = data.cuda(), labels.cuda()

    t0 = time.time()
    embed = model(g, node_id, edge_type, edge_norm)
    print(embed.shape)
    loss = model.get_loss(g, embed, data, labels)
    t1 = time.time()
    loss.backward()
    # grade norm = 1.0
    torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) # clip gradients
    optimizer.step()
    t2 = time.time()

    forward_time.append(t1 - t0)
    backward_time.append(t2 - t1)
    if epoch % 25 == 0:
        print("Epoch {:04d} | Loss {:.4f} | Best MRR {:.4f} | Forward {:.4f}s | Backward {:.4f}s".
          format(epoch, loss.item(), best_mrr, forward_time[-1], backward_time[-1]))

    optimizer.zero_grad()

    # validation
    if epoch % 100 == 0:
        # gc.collect()
        # perform validation on CPU because full graph is too large
        if use_cuda:
            model.cpu()
        model.eval()
        print("start eval")
        print(test_graph)
        # if use_cuda:
        #     test_graph = test_graph.to('cuda:0')
        #     test_node_id, test_rel, test_norm = test_node_id.cuda(), test_rel.cuda(), test_norm.cuda()
        # embed = model.embedding.embedding
        # embed = embed(test_node_id)
        embed = model(test_graph, test_node_id, test_rel, test_norm)
        # torch.cuda.empty_cache()
        # gc.collect()
        # args.eval_batch_size = 500
        mrr = utils.calc_mrr(embed, model.w_relation, test_data,
                             hits=[1, 3, 10], eval_bz=500)
        # save best model
        if mrr < best_mrr:
            # min train epoch = 6000
            if epoch >= 6000:
                break
        else:
            best_mrr = mrr
            torch.save({'state_dict': model.state_dict(), 'epoch': epoch},
                       model_state_file)
        if use_cuda:
            model.cuda()

torch.Size([736, 768])
torch.Size([706, 768])
torch.Size([699, 768])
torch.Size([697, 768])
torch.Size([679, 768])
torch.Size([733, 768])
torch.Size([719, 768])
torch.Size([723, 768])
torch.Size([719, 768])
torch.Size([684, 768])
torch.Size([693, 768])
torch.Size([684, 768])
torch.Size([715, 768])
torch.Size([689, 768])
torch.Size([706, 768])
torch.Size([706, 768])
torch.Size([689, 768])
torch.Size([692, 768])
torch.Size([736, 768])
torch.Size([679, 768])
torch.Size([699, 768])
torch.Size([693, 768])
torch.Size([712, 768])
torch.Size([725, 768])
torch.Size([702, 768])
Epoch 0025 | Loss 3470.8987 | Best MRR 0.0000 | Forward 0.0163s | Backward 0.0262s
torch.Size([715, 768])
torch.Size([690, 768])
torch.Size([708, 768])
torch.Size([695, 768])
torch.Size([710, 768])
torch.Size([680, 768])
torch.Size([726, 768])
torch.Size([711, 768])
torch.Size([703, 768])
torch.Size([708, 768])
torch.Size([697, 768])
torch.Size([711, 768])
torch.Size([703, 768])
torch.Size([721, 768])
torch.Size([724, 768

KeyboardInterrupt: 